In [1]:
# coding: utf-8
# Our base common modules
from Common.UtilFuncs import Evaluator, Lab

# Classifiers without denoising
import Common.SvmMethod as SvmMethod

# Denoising Methodes
import Common.IsolationForestMethod as IsolationForestMethod
import Common.ConfidentLearningMethod as ConfidentLearningMethod
import Common.LocalOutlierFactorMethod as LocalOutlierFactorMethod
from Common.BERTModel import do_experiment_BERT, BERTModel

In [2]:
# The settings of the noise sources.
# Each item: source -> (size, distribution)
noisy_set_sizes0 = {
    'mislabeled' : (7600, None),                   # max size: 15000
    # 'irrelevant' : (8600, [0.25,0.25,0.25,0.25]),  # max size: 34259
    # 'translated' : (8600, "reserve_labels"),       # max size: 5000
}

noisy_set_sizes1 = {
    # 'mislabeled' : (8600, None),                   # max size: 15000
    'irrelevant' : (8600, [0.25,0.25,0.25,0.25]),  # max size: 34259
    # 'translated' : (8600, "reserve_labels"),       # max size: 5000
}

noisy_set_sizes2 = {
    # 'mislabeled' : (8600, None),                   # max size: 15000
    # 'irrelevant' : (8600, [0.25,0.25,0.25,0.25]),  # max size: 34259
    'translated' : (8600, 0),       # max size: 5000
}

noisy_set_sizes3 = {
    # 'mislabeled' : (8600, None),                   # max size: 15000
    # 'irrelevant' : (8600, [0.25,0.25,0.25,0.25]),  # max size: 34259
    'translated' : (8600, 0.50),                   # max size: 5000
}

noisy_set_sizes4 = {
    # 'mislabeled' : (8600, None),                   # max size: 15000
    # 'irrelevant' : (8600, [0.25,0.25,0.25,0.25]),  # max size: 34259
    'translated' : (8600, 1),                   # max size: 5000
}

# Choose a experiment without denoising
# Each item: name -> (funcion, whether choose) note:only the first active one will be used
experiment_without_denoising = {
    'SVM' : (SvmMethod.do_experiment, 0),
    'BERT' : (do_experiment_BERT, 1),
}

# Choose a experiment with denoising
# Each item: name -> (funcion, whether choose) note:only the first active one will be used
# experiment_with_denoising = {
#     'Confident Learning' : (do_experiment_denoised_by_ConfidentLearning, 1),
#     'Isolation Forest'   : (do_experiment_denoised_by_IsolationForest,   0),
# }

# The training set of each experiment
#origin_train_set_sizes = [2000, 4000, 5000, 8000, 10000, 15000, 20000]
origin_train_set_sizes = [2000]
# origin_train_set_sizes = [5000, 10000, 15000, 20000]
noisy_train_set_sizes  = [ ]#(12000,3000)]
# noisy_train_set_sizes  = [(1000, 2000)]

In [3]:



RUN = 1      #1/0:  Run new experiments / Read results made by previous experiments


if RUN:
    # Run new experiments
    # Initialize the lab, which will run a serial of experiments

    lab = Lab("twitter_sentiment_data_clean.csv", noisy_sources = noisy_set_sizes0, total_train_size = 20000, total_test_size = 4000, validation_size=2000)
    experiment_without_denoising['BERT'] = (do_experiment_BERT, (lab,), 1)
    lab.set_experiment_no_denoising(experiment_without_denoising)

    # Run experiment on original data
    lab.do_batch_experiments(origin_train_set_sizes)

    # Run experiment on noisy data -- mislabed noise
    lab.set_noisy_sources(noisy_set_sizes0)
    lab.do_batch_experiments(noisy_train_set_sizes)

    # Run experiment on noisy data -- irrelevant noise
    lab.set_noisy_sources(noisy_set_sizes1)
    lab.do_batch_experiments(noisy_train_set_sizes)

    # Run experiment on noisy data -- translated noise
    lab.set_noisy_sources(noisy_set_sizes2)
    lab.do_batch_experiments(noisy_train_set_sizes)

    # Run experiment on noisy data -- mislabeled translated noise
    lab.set_noisy_sources(noisy_set_sizes3)
    lab.do_batch_experiments(noisy_train_set_sizes)

    # Run experiment on noisy data -- part-mislabeled translated noise
    lab.set_noisy_sources(noisy_set_sizes4)
    lab.do_batch_experiments(noisy_train_set_sizes)

    # Save the results
    lab.save("saving/noise_effect.pk")
else:
    # Read evaluations saved by previous experiments
    lab = Lab.load("saving/noise_effect.pk")

# Show evaluations in a form
lab.print()

# Plot the evaluations
# lab.plot()

for i in range(2):
    # Plot training set size vs. Macro F1

    # x coordinate
    if i == 0:
        xValue  = "x['Origin']+x['Noise']"
        xLabel  = "Training set total size\nnoisy sets: %s" % \
                    str([str(x[0])+'+'+str(x[1]) for x in noisy_train_set_sizes]).replace("\'","")
    else:
        xValue  = "x['Origin']"
        xLabel  = "Training set origin part size\nnoisy sets: %s" % \
                    str([str(x[0])+'+'+str(x[1]) for x in noisy_train_set_sizes]).replace("\'","")

    # y coordinate
    yValue  = "y['Macro F1']"

    # Divide experiments into several groups, each will be plotted as a line
    len1 = len(origin_train_set_sizes)
    len2 = len(noisy_train_set_sizes)
    lines = { # each item: name, filter
        'Original Data':       "int((df['Experiment']-1)/%d)==0"%len1,
        'Mislabeled Noise':    "int((df['Experiment']-1-%d)/%d)==0 and df['Experiment']-1-%d>=0"%(len1,len2,len1),
        'Irrelevant Noise':    "int((df['Experiment']-1-%d)/%d)==1"%(len1,len2),
        'Translated Noise(0% mislabeled)':    "int((df['Experiment']-1-%d)/%d)==2"%(len1,len2),
        'Translated Noise(50% mislabeled)':   "int((df['Experiment']-1-%d)/%d)==3"%(len1,len2),
        'Translated Noise(100% mislabeled)':  "int((df['Experiment']-1-%d)/%d)==4"%(len1,len2),
    }

    # Do plot
    lab.Ev.plot(xValue = xValue, yValue = yValue, lines = lines,
                xLabel = xLabel, title = "SVM effected by various noises")



* 1> Training set size: 2000 samples
  Sentiments ('Anti', 'Neutral', 'Pro', 'News'): 9.4%, 18.3%, 50.2%, 22.1%
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-22 23:53:29.205320: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-22 23:53:29.205505: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-22 23:53:29.225887: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


  0%|          | 0/2000 [00:00<?, ?it/s]/Users/guopei/miniforge3/envs/tfgpu/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2104: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 1000/1000 [00:00<00:00, 3395.30it/s]


Epoch 1/50


...

2022-04-22 23:53:36.712378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - ETA: 0s - loss: 1.1879 - accuracy: 0.5175WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


2022-04-22 23:59:19.142756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 395s 3s/step - loss: 1.1879 - accuracy: 0.5175 - val_loss: 0.9470 - val_accuracy: 0.6080 - lr: 1.0000e-04
Epoch 2/50
105/125 [========================>.....] - ETA: 1:01 - loss: 0.9611 - accuracy: 0.5952

KeyboardInterrupt: 

In [ ]:
BERTModel.sInstance.PrintAccuracy()

In [ ]:
BERTModel.sInstance.PrintLoss()